In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

np.random.seed(37)

df = pd.read_csv('./faces/data.csv')
X = df[[i for i in df.columns if i != 'y']]
y = df['y']

In [11]:
X.shape

(550, 18)

In [12]:
y.shape

(550,)

In [15]:
y

0      0
1      0
2      0
3      0
4      0
      ..
545    3
546    3
547    3
548    3
549    3
Name: y, Length: 550, dtype: int64